# TEMPLATE TRAIN



## Globals Params

In [ ]:
## run params
batchSize   = 256
numWork     = 2 #<! Number of workers
nEpochs     = 2

## import
* use armo include for all general includes

In [ ]:
import sys
sys.path.append('../')
from armoInclude import *

In [ ]:
# Specific tasks:

from armoTrain import basicModel

## get the data

In [ ]:
dsTrain, dsVal ,dsTest = ## get the data....

#### split methods


In [ ]:
mXTrain, mXTest, vYTrain, vYTest = train_test_split(mX, vY, train_size = trainRatio, test_size = testRatio, random_state = seedNum)

# Split data to train and validation:
oDataSet            = torch.utils.data.TensorDataset(mX, vF)
dsTrain, dsVal  = torch.utils.data.random_split(dsData, [numSamplesTrain, numSamplesVal])


## data process

####  Standardization

In [ ]:
# Calculate the Standardization Parameters
vMean = np.mean(dsTrain.data / 255.0, axis = (0, 1, 2))
vStd  = np.std(dsVal.data / 255.0, axis = (0, 1, 2))
print('µ =', vMean)
print('σ =', vStd)

### Transforms

In [ ]:
## V1
oDataTrns = torchvision.transforms.Compose([ 
    torchvision.transforms.ToTensor(),        
    torchvision.transforms.Normalize(µ, σ),  
    ])
# Update the DS transformer
dsTrain.transform = oDataTrns
dsVal.transform     = oDataTrns
dsTest.transform  = oDataTrns

## V2
oDataTrns = TorchVisionTrns.Compose([
    TorchVisionTrns.ToImage(),
    TorchVisionTrns.ToDtype(torch.float32, scale = True),
    TorchVisionTrns.Normalize(mean = vMean, std = vStd),
])
# Update the DS transformer
dsTrain.transform   = oDataTrns
dsVal.transform     = oDataTrns
dsTest.transform     = oDataTrns

### data loaders

In [ ]:
dlTrain = torch.utils.data.DataLoader(dsTrain, shuffle = True, batch_size = 1 * batchSize, num_workers = numWork, persistent_workers = True)
dlVal   = torch.utils.data.DataLoader(dsVal, shuffle = False, batch_size = 2 * batchSize, num_workers = numWork, persistent_workers = True)
dlTest   = torch.utils.data.DataLoader(dsTest, shuffle = False, batch_size = 2 * batchSize, num_workers = numWork, persistent_workers = True)

In [ ]:
# Iterate on the Loader
tX, vY = next(iter(dlTrain)) #<! PyTorch Tensors
print(f'The batch features dimensions: {tX.shape}')
print(f'The batch labels dimensions: {vY.shape}')

## layers

In [ ]:
class CustomLAYER():
    def __init__( self ) -> None:
        self.mX = None #<! Required for the backward pass
        self.dGrads = {}
    
    def Forward( self: Self, mX: np.ndarray ) -> np.ndarray:
        self.mX = mX                 #<! Store for Backward
        mZ      = ????
        return mZ
    
    def Backward( self: Self, mDz: np.ndarray ) -> np.ndarray:
        mX    = self.mX
        mDx = ????
        return mDx

In [ ]:
class CustomBlock( nn.Module ):
    def __init__( self, numChnl: int ) -> None:
        super(CustomBlock, self).__init__()
        
        self.oConv2D1       = nn.Conv2d(...
        self.oBatchNorm1    = nn.BatchNorm2d(...
        self.oReLU1         = nn.ReLU(...
        self.CustomLAYER    = CustomLAYER(...
            
    def forward( self: Self, tX: torch.Tensor ) -> torch.Tensor:
        
        tY = ...
        tY += ...
		
        return tY

In [ ]:
def BuildModel( nC: int ) -> nn.Module:
    
    oModel = nn.Sequential(

        nn.Identity(),

        nn.Conv2d(...
        
        CustomBlock(...
        
    )

    return oModel

In [ ]:
oModel = BuildModel(len(L_CLASSES))

## model

In [ ]:
myTrain = basicModel(model=oModel, debug=True)


#### target HW

In [ ]:
runDevice = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') #<! The 1st CUDA device
oModel = oModel.to(runDevice)

### loss

In [ ]:
class ObjLocLoss( nn.Module ):
    def __init__( self, numCls: int, λ: float, ϵ: float = 0.0 ) -> None:
        super(ObjLocLoss, self).__init__()
        self.numCls     = numCls
    
    def forward( self: Self, mYHat: torch.Tensor, mY: torch.Tensor ) -> torch.Tensor:
        ## 
        return lossVal

hL = ObjLocLoss(numCls = len(L_CLASSES), λ = λ, ϵ = ϵ)
hL = hL.to(runDevice)


### score

In [ ]:
class ObjLocScore( nn.Module ):
    def __init__( self, numCls: int ) -> None:
        super(ObjLocScore, self).__init__()
        self.numCls = numCls
    
    def forward( self: Self, mYHat: torch.Tensor, mY: torch.Tensor ) -> Tuple[float, float, float]:
        batchSize = mYHat.shape[0]
        ## 
        return valScore

hS = ObjLocScore(numCls = len(L_CLASSES))
hS = hS.to(runDevice)


### optimizer

In [ ]:
oOpt = torch.optim.AdamW(oModel.parameters(), lr = 1e-5, betas = (0.9, 0.99), weight_decay = 1e-5) 

### scheduler

In [ ]:
oSch = torch.optim.lr_scheduler.OneCycleLR(oOpt, max_lr = 5e-4, total_steps = numEpochs)


### summary

In [ ]:
myTrain.summary(dlTrain)

### verify

In [ ]:
myTrain.verify( dlTrain, dlVal, oOpt, nEpochs , batchSize , hL, hS, oSch = oSch)


### train

In [ ]:
trained = myTrain.TrainModel(dlTrain, dlVal, oOpt, nEpochs, hL, hS, oSch = oSch)

## EVAL

### plot results

In [ ]:
myTrain.plotTrainResults()

### eval test data

In [ ]:
myTrain.evaluate_test_data(testData = dlTest ,loss_fn=hL)

### inference

In [ ]:
test_x_data = mXTest.view(numSignalsTest, 1, -1)
eval_result = myTrain.inference(test_x_data)
print(f"test_x_data[1] = {test_x_data[1] }")      
print(f"eval_result[1] = {eval_result[1] }")    